Remove Existing Table and Create New One

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_values

### Create the table with all the column names
#-- Drop Table ris_table;

db_config = {
    "dbname": "web_application_production",
    "user": "april",
    "password": "u&e!!!s4g3es28iTv3oqvkBod",
    "host": "localhost",
    "port": "5432"
}


create_table_sql = """
DROP TABLE IF EXISTS ris_sheet;

CREATE TABLE ris_sheet (
    id SERIAL PRIMARY KEY,
    plan_contact_event TEXT,
    img_time_utc TEXT,
    target TEXT,
    latitude TEXT,
    longitude TEXT,
    expected_cloud_cover_percent TEXT,
    slow_down TEXT,
    frame_rate_mhz TEXT,
    tilt_angle_degree TEXT,
    encryption TEXT,
    compression TEXT,
    rec_1 TEXT,
    exposure_microsec_1 TEXT,
    focm_1 TEXT,
    htr_temperature_c_1 TEXT,
    binning_1 TEXT,
    confirmed_1 TEXT,
    type_of_acquisition_1 TEXT,
    type_of_download_1 TEXT,
    tx_date_1 TEXT,
    cadu_file_name_1 TEXT,
    cadu_file_size_gb_1 TEXT,
    planned_frames_1 TEXT,
    downloaded_1 TEXT,
    pnms_1 TEXT,
    comments_1 TEXT,
    repeated_frames_1 TEXT,
    loss_of_frames_1 TEXT,
    rec_2 TEXT,
    exposure_2 TEXT,
    focm_2 TEXT,
    htr_temperature_c_2 TEXT,
    binning_2 TEXT,
    confirmed_2 TEXT,
    type_of_acquisition_2 TEXT,
    type_of_download_2 TEXT,
    tx_date_2 TEXT,
    cadu_file_name_2 TEXT,
    cadu_file_size_gb_2 TEXT,
    planned_frames_2 TEXT,
    downloaded_2 TEXT,
    pnms_2 TEXT,
    comments_2 TEXT,
    repeated_frames_2 TEXT,
    loss_of_frames_2 TEXT
);
"""
# create_table_sql = """
# DROP TABLE IF EXISTS ris_sheet;

# CREATE TABLE ris_sheet (
#     "id" SERIAL PRIMARY KEY,
#     "Plan Contact Event" TEXT,
#     "IMG TIME (UTC)" TEXT,
#     "TARGET" TEXT,
#     "LATITUDE" TEXT,
#     "LONGITUDE" TEXT,
#     "Expected Cloud Cover (%)" TEXT,
#     "Slow Down (1 to 5)" TEXT,
#     "Frame Rate (mHz)" TEXT,
#     "Tilt Angle (degree)" TEXT,
#     "Encryption (1=ON, 0=OFF)" TEXT,
#     "Compression (1=ON, 0=OFF)" TEXT,
#     "1REC" TEXT,
#     "1Exposure_(microsec)" TEXT,
#     "1FOCM" TEXT,
#     "1HTR Temperature (Deg. C)" TEXT,
#     "1Binning" TEXT,
#     "1CONFIRMED" TEXT,
#     "1TYPE OF ACQUISITION" TEXT,
#     "1TYPE OF DOWNLOAD" TEXT,
#     "1TX DATE" TEXT,
#     "1CADU File Name" TEXT,
#     "1CADU File Size (GB)" TEXT,
#     "1PLANNED_FRAMES" TEXT,
#     "1DOWNLOADED" TEXT,
#     "1PNMs" TEXT,
#     "1COMMENTS" TEXT,
#     "1Repeated Frames" TEXT,
#     "1Loss of Frames" TEXT,
#     "2REC" TEXT,
#     "2Exposure" TEXT,
#     "2FOCM" TEXT,
#     "2HTR Temperature (Deg. C)" TEXT,
#     "2Binning" TEXT,
#     "2CONFIRMED" TEXT,
#     "2TYPE OF ACQUISITION" TEXT,
#     "2TYPE OF DOWNLOAD" TEXT,
#     "2TX DATE" TEXT,
#     "2CADU File Name" TEXT,
#     "2CADU File Size (GB)" TEXT,
#     "2PLANNED_FRAMES" TEXT,
#     "2DOWNLOADED" TEXT,
#     "2PNMs" TEXT,
#     "2COMMENTS" TEXT,
#     "2Repeated Frames" TEXT,
#     "2Loss of Frames" TEXT
# );
# """
conn = psycopg2.connect(
        host="localhost",
        port="5432",
        database="web_application_production",
        user="april",
        password="u&e!!!s4g3es28iTv3oqvkBod"
    )
with conn.cursor() as cursor:
    cursor.execute("""
        SELECT EXISTS (
            SELECT FROM information_schema.tables 
            WHERE table_name = 'ris_sheet'
        );
    """)
    print(cursor.fetchone()[0])

# Check columns
with conn.cursor() as cursor:
    cursor.execute("""
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = 'ris_sheet';
    """)
    print([col[0] for col in cursor.fetchall()])

cursor = conn.cursor()
cursor.execute(create_table_sql)

print("Database Table ris_sheet successfully.")




# File path to the Excel file
file_path = '/home/pavan_azista/Downloads/AFR_Spacecraft_Status_Overview.xlsx'


# Load the data from the Excel file
data = pd.read_excel(file_path)

data_table = data.iloc[4:].reset_index(drop=True)
data_table.columns = data_table.iloc[0]  # Use the 5th row as column names
data_table = data_table[1:]

# data_table.columns = [str(col).strip() for col in data_table.columns]
data_table.columns = [str(col).strip() for col in data_table.columns]

for col in data_table.columns:
    if pd.api.types.is_datetime64_any_dtype(data_table[col]):  # Check if column is datetime
        data_table[col] = data_table[col].dt.strftime('%d-%m-%Y %H:%M:%S') 



data_table = data_table.iloc[::-1].reset_index(drop=True)
# data_table = data_table.replace({np.nan: None})
data_table = data_table.replace({np.nan: "-"})  



data_tuples = data.values.tolist()

print("Sample Data:", data_tuples[:2])
csv_file_path = '/home/pavan_azista/Documents/RIS_sheet.csv'
data_table.to_csv(csv_file_path, sep='|', index=False)

with open(csv_file_path, 'r', encoding='utf-8') as f:
    # cursor.copy_expert(f"""
    #     COPY public.ris_sheet (
    #         "Plan Contact Event", "IMG TIME (UTC)", "TARGET", "LATITUDE", "LONGITUDE",
    #         "Expected Cloud Cover (%)", "Slow Down (1 to 5)", "Frame Rate (mHz)",
    #         "Tilt Angle (degree)", "Encryption (1=ON, 0=OFF)", "Compression (1=ON, 0=OFF)",
    #         "1REC", "1Exposure_(microsec)", "1FOCM", "1HTR Temperature (Deg. C)",
    #         "1Binning", "1CONFIRMED", "1TYPE OF ACQUISITION", "1TYPE OF DOWNLOAD",
    #         "1TX DATE", "1CADU File Name", "1CADU File Size (GB)", "1PLANNED_FRAMES",
    #         "1DOWNLOADED", "1PNMs", "1COMMENTS", "1Repeated Frames", "1Loss of Frames",
    #         "2REC", "2Exposure", "2FOCM", "2HTR Temperature (Deg. C)", "2Binning",
    #         "2CONFIRMED", "2TYPE OF ACQUISITION", "2TYPE OF DOWNLOAD", "2TX DATE",
    #         "2CADU File Name", "2CADU File Size (GB)", "2PLANNED_FRAMES", "2DOWNLOADED", 
    #         "2PNMs", "2COMMENTS", "2Repeated Frames", "2Loss of Frames"
    #     ) FROM STDIN WITH DELIMITER '|' CSV HEADER ENCODING 'UTF8' QUOTE '\"' ESCAPE '\"';
    # """, f)
    cursor.copy_expert(f"""
        COPY public.ris_sheet (
            "plan_contact_event","img_time_utc","target","latitude","longitude",
            "expected_cloud_cover_percent","slow_down","frame_rate_mhz",
            "tilt_angle_degree","encryption","compression","rec_1",
            "exposure_microsec_1","focm_1","htr_temperature_c_1",
            "binning_1","confirmed_1","type_of_acquisition_1",
            "type_of_download_1","tx_date_1","cadu_file_name_1",
            "cadu_file_size_gb_1","planned_frames_1","downloaded_1",
            "pnms_1","comments_1","repeated_frames_1","loss_of_frames_1",
            "rec_2","exposure_2","focm_2","htr_temperature_c_2","binning_2",
            "confirmed_2","type_of_acquisition_2","type_of_download_2","tx_date_2",
            "cadu_file_name_2","cadu_file_size_gb_2","planned_frames_2","downloaded_2",
            "pnms_2","comments_2","repeated_frames_2","loss_of_frames_2"
        ) FROM STDIN WITH DELIMITER '|' CSV HEADER ENCODING 'UTF8' QUOTE '\"' ESCAPE '\"';
    """, f)
conn.commit()
cursor.close()
conn.close()
print("Data has been ingested into the ris_sheet table successfully.")


True
['id', 'plan_contact_event', 'img_time_utc', 'target', 'latitude', 'longitude', 'expected_cloud_cover_percent', 'slow_down', 'frame_rate_mhz', 'tilt_angle_degree', 'encryption', 'compression', 'rec_1', 'exposure_microsec_1', 'focm_1', 'htr_temperature_c_1', 'binning_1', 'confirmed_1', 'type_of_acquisition_1', 'type_of_download_1', 'tx_date_1', 'cadu_file_name_1', 'cadu_file_size_gb_1', 'planned_frames_1', 'downloaded_1', 'pnms_1', 'comments_1', 'repeated_frames_1', 'loss_of_frames_1', 'rec_2', 'exposure_2', 'focm_2', 'htr_temperature_c_2', 'binning_2', 'confirmed_2', 'type_of_acquisition_2', 'type_of_download_2', 'tx_date_2', 'cadu_file_name_2', 'cadu_file_size_gb_2', 'planned_frames_2', 'downloaded_2', 'pnms_2', 'comments_2', 'repeated_frames_2', 'loss_of_frames_2']


SyntaxError: syntax error at or near "#"
LINE 52: # 
         ^


Upload Data into the Table from an XLSX

In [ ]:




# columns = ', '.join([f'"{col}"' for col in data_table.columns])  # Ensure proper quoting

# values_placeholders = ', '.join(['%s' for _ in data_table.columns])

# insert_query = f'INSERT INTO ris_sheet ({columns}) VALUES ({values_placeholders})'



# If the column names in the Excel file are already correct and match the table,
# then there's no need to rename them. If they don't, you can rename the columns
# as needed, for example:
# data.columns = ["Plan Contact Event", "IMG TIME (UTC)", "TARGET", ...]

# Connect to the PostgreSQL database

# Define the insert query for the table
# insert_query = """
#     INSERT INTO ris_sheet (
#         "Plan Contact Event", "IMG TIME (UTC)", "TARGET", "LATITUDE", "LONGITUDE",
#         "Expected Cloud Cover (%)", "Slow Down (1 to 5)", "Frame Rate (mHz)", "Tilt Angle (degree)",
#         "Encryption (1=ON, 0=OFF)", "Compression (1=ON, 0=OFF)", "1REC", "1Exposure_(microsec)", 
#         "1FOCM", "1HTR Temperature (Deg. C)", "1Binning", "1CONFIRMED", "1TYPE OF ACQUISITION", 
#         "1TYPE OF DOWNLOAD", "1TX DATE", "1CADU File Name", "1CADU File Size (GB)", "1PLANNED_FRAMES", 
#         "1DOWNLOADED", "1PNMs", "1COMMENTS", "1Repeated Frames", "1Loss of Frames", "2REC", 
#         "2Exposure", "2FOCM", "2HTR Temperature (Deg. C)", "2Binning", "2CONFIRMED", 
#         "2TYPE OF ACQUISITION", "2TYPE OF DOWNLOAD", "2TX DATE", "2CADU File Name", 
#         "2CADU File Size (GB)", "2PLANNED_FRAMES", "2DOWNLOADED", "2PNMs", "2COMMENTS", 
#         "2Repeated Frames", "2Loss of Frames"
#     ) VALUES %s
# """
# insert_query = f"""
#     INSERT INTO ris_sheet ({", ".join(f'"{col}"' for col in data_table.columns)})
#     VALUES %s
# """


# execute_values(cursor, insert_query, data_tuples)

# Use execute_values for efficient bulk insert


Sample Data: [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]
Data has been ingested into the ris_sheet table successfully.


/home/pavan_azista/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/home/pavan_azista/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():


In [6]:
csv_file_path = '/home/pavan_azista/Documents/RIS_sheet.csv'



conn = psycopg2.connect(
        host="localhost",
        port="5432",
        database="web_application_production",
        user="april",
        password="u&e!!!s4g3es28iTv3oqvkBod"
    )
cursor = conn.cursor()

with open(csv_file_path, 'r', encoding='utf-8') as f:
    cursor.copy_expert(f"""
        COPY public.ris_sheet (
            "Plan Contact Event", "IMG TIME (UTC)", "TARGET", "LATITUDE", "LONGITUDE",
            "Expected Cloud Cover (%)", "Slow Down (1 to 5)", "Frame Rate (mHz)",
            "Tilt Angle (degree)", "Encryption (1=ON, 0=OFF)", "Compression (1=ON, 0=OFF)",
            "1REC", "1Exposure_(microsec)", "1FOCM", "1HTR Temperature (Deg. C)",
            "1Binning", "1CONFIRMED", "1TYPE OF ACQUISITION", "1TYPE OF DOWNLOAD",
            "1TX DATE", "1CADU File Name", "1CADU File Size (GB)", "1PLANNED_FRAMES",
            "1DOWNLOADED", "1PNMs", "1COMMENTS", "1Repeated Frames", "1Loss of Frames",
            "2REC", "2Exposure", "2FOCM", "2HTR Temperature (Deg. C)", "2Binning",
            "2CONFIRMED", "2TYPE OF ACQUISITION", "2TYPE OF DOWNLOAD", "2TX DATE",
            "2CADU File Name", "2CADU File Size (GB)", "2PLANNED_FRAMES", "2DOWNLOADED", 
            "2PNMs", "2COMMENTS", "2Repeated Frames", "2Loss of Frames"
        ) FROM STDIN WITH DELIMITER '|' CSV HEADER ENCODING 'UTF8' QUOTE '\"' ESCAPE '\"';
    """, f)
# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

BadCopyFileFormat: extra data after last expected column
CONTEXT:  COPY ris_sheet, line 2: "0|||Launcher Integration|||||||||1r74||||1|DELETED|||||||25||DELETED|||2r47||||1|DELETED|||||||||||"


In [ ]:

# Execute the SQL command to drop and create the table



# Load the data
# file_path = '/home/pavan_azista/Downloads/AFR_Spacecraft_Status_Overview.xlsx'
# # data = pd.read_excel(file_path, engine='odf')
# data = pd.read_excel(file_path)

# Clean and structure the data (row 5 as headers)
# data_table = data.iloc[4:].reset_index(drop=True)
# data_table.columns = data_table.iloc[0]  # Use the 5th row as column names
# data_table = data_table[1:]  # Exclude the header row

# # Select the specific 45 columns
# columns_to_consider = [
#     "Plan Contact Event", "IMG TIME (UTC)", "TARGET", "LATITUDE", "LONGITUDE",
#     "Expected Cloud Cover (%)", "Slow Down (1 to 5)", "Frame Rate (mHz)", "Tilt Angle (degree)",
#     "Encryption (1=ON, 0=OFF)", "Compression (1=ON, 0=OFF)", "1REC", "1Exposure_(microsec)", 
#     "1FOCM", "HTR Temperature (Deg. C)", "1Binning", "1CONFIRMED", "1TYPE OF ACQUISITION", 
#     "1TYPE OF DOWNLOAD", "1TX DATE", "1CADU File Name", "1CADU File Size (GB)", "1PLANNED_FRAMES", 
#     "1DOWNLOADED", "1PNMs", "1COMMENTS", "1Repeated Frames", "1Loss of Frames", "2REC", 
#     "2Exposure", "2FOCM", "2HTR Temperature (Deg. C)", "2Binning", "2CONFIRMED", 
#     "2TYPE OF ACQUISITION", "2TYPE OF DOWNLOAD", "2TX DATE", "2CADU File Name", 
#     "2CADU File Size (GB)", "2PLANNED_FRAMES", "2DOWNLOADED", "2PNMs", "2COMMENTS", 
#     "2Repeated Frames", "2Loss of Frames"
# ]
# data_table = data_table[columns_to_consider]

# # Rename columns for SQL compatibility
# data_table.columns = [
#     "Plan Contact Event", "IMG TIME (UTC)", "TARGET", "LATITUDE", "LONGITUDE",
#     "Expected Cloud Cover (%)", "Slow Down (1 to 5)", "Frame Rate (mHz)", "Tilt Angle (degree)",
#     "Encryption (1=ON, 0=OFF)", "Compression (1=ON, 0=OFF)", "1REC", "1Exposure_(microsec)", 
#     "1FOCM", "1HTR Temperature (Deg. C)", "1Binning", "1CONFIRMED", "1TYPE OF ACQUISITION", 
#     "1TYPE OF DOWNLOAD", "1TX DATE", "1CADU File Name", "1CADU File Size (GB)", "1PLANNED_FRAMES", 
#     "1DOWNLOADED", "1PNMs", "1COMMENTS", "1Repeated Frames", "1Loss of Frames", "2REC", 
#     "2Exposure", "2FOCM", "2HTR Temperature (Deg. C)", "2Binning", "2CONFIRMED", 
#     "2TYPE OF ACQUISITION", "2TYPE OF DOWNLOAD", "2TX DATE", "2CADU File Name", 
#     "2CADU File Size (GB)", "2PLANNED_FRAMES", "2DOWNLOADED", "2PNMs", "2COMMENTS", 
#     "2Repeated Frames", "2Loss of Frames"
# ]
# Database connection settings
# local_conn_1 = psycopg2.connect(
#     # host="52.66.31.186",
#     host="localhost",
#     port="5432",
#     database="web_application",
#     user="april",
#     password="u&e!!!s4g3es28iTv3oqvkBod"
# )
# Connect to PostgreSQL

# # Insert data into the table
# insert_query = """
#     INSERT INTO ris_sheet (
#     "Plan Contact Event", "IMG TIME (UTC)", "TARGET", "LATITUDE", "LONGITUDE",
#     "Expected Cloud Cover (%)", "Slow Down (1 to 5)", "Frame Rate (mHz)", "Tilt Angle (degree)",
#     "Encryption (1=ON, 0=OFF)", "Compression (1=ON, 0=OFF)", "1REC", "1Exposure_(microsec)", 
#     "1FOCM", "1HTR Temperature (Deg. C)", "1Binning", "1CONFIRMED", "1TYPE OF ACQUISITION", 
#     "1TYPE OF DOWNLOAD", "1TX DATE", "1CADU File Name", "1CADU File Size (GB)", "1PLANNED_FRAMES", 
#     "1DOWNLOADED", "1PNMs", "1COMMENTS", "1Repeated Frames", "1Loss of Frames", "2REC", 
#     "2Exposure", "2FOCM", "2HTR Temperature (Deg. C)", "2Binning", "2CONFIRMED", 
#     "2TYPE OF ACQUISITION", "2TYPE OF DOWNLOAD", "2TX DATE", "2CADU File Name", 
#     "2CADU File Size (GB)", "2PLANNED_FRAMES", "2DOWNLOADED", "2PNMs", "2COMMENTS", 
#     "2Repeated Frames", "2Loss of Frames"
#     ) VALUES %s
# """

# data_table = data_table.iloc[::-1].reset_index(drop=True)

# data_table = data_table.replace({np.nan: None})

# # Convert the DataFrame to a list of tuples
# data_tuples = data_table.values.tolist()

# Use execute_values for efficient bulk insert
# execute_values(cursor, insert_query, data_tuples)

# Commit and close the connection

